<a href="https://colab.research.google.com/github/OmarAyman239/STAI-Deepfake/blob/main/STAI_Proj_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing necessary libraries

In [ ]:
# Required Libraries
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix

# PyTorch for AlexNet
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset

# TensorFlow for VGG16
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
import kagglehub

##Loading the dataset and checking its path

In [ ]:
# Download latest version
path = kagglehub.dataset_download("vijaydevane/deepfake-detection-challenge-dataset-face-images")

print("Path to dataset files:", path)

# Check downloaded files
print(os.listdir(path))

# Check the 'real' and 'fake' folders inside
real_path = os.path.join(path, "real")
fake_path = os.path.join(path, "fake")

print("Real folder contents:", os.listdir(real_path))
print("Fake folder contents:", os.listdir(fake_path))


Path to dataset files: /root/.cache/kagglehub/datasets/vijaydevane/deepfake-detection-challenge-dataset-face-images/versions/1
['real', 'fake']
Real folder contents: ['bulkxhhknf_2.png', 'bzythlfnhq_0.png', 'dlpoieqvfb_4.png', 'eudeqjhdfd_7.png', 'ehtdtkmmli_7.png', 'dzyuwjkjui_2.png', 'axntxmycwd_1.png', 'aytzyidmgs_8.png', 'bilnggbxgu_1.png', 'aelfnikyqj_0.png', 'bmjzrlszhi_4.png', 'aytzyidmgs_3.png', 'eggbjzxnmg_4.png', 'dbnygxtwek_6.png', 'axntxmycwd_8.png', 'aybumesmpk_7.png', 'brwrlczjvi_5.png', 'dkuayagnmc_9.png', 'dakiztgtnw_3.png', 'asaxgevnnp_1.png', 'ellavthztb_7.png', 'bffwsjxghk_2.png', 'atkdltyyen_3.png', 'bddjdhzfze_9.png', 'ellavthztb_4.png', 'drcyabprvt_6.png', 'eqnoqyfquo_2.png', 'bffwsjxghk_3.png', 'agrmhtjdlk_2.png', 'anpuvshzoo_6.png', 'bdnaqemxmr_2.png', 'bulkxhhknf_8.png', 'dsjbknkujw_9.png', 'cpjxareypw_2.png', 'anpuvshzoo_7.png', 'bzythlfnhq_3.png', 'cppdvdejkc_8.png', 'dzyuwjkjui_8.png', 'asaxgevnnp_9.png', 'crezycjqyk_8.png', 'ccfoszqabv_8.png', 'ckkuyewywx_5

##Load Images

In [ ]:
from PIL import Image
import glob

# Load real images
real_images = [Image.open(img) for img in glob.glob(f"{real_path}/*")]

# Load fake images
fake_images = [Image.open(img) for img in glob.glob(f"{fake_path}/*")]

print(f"Loaded {len(real_images)} real images and {len(fake_images)} fake images.")

Loaded 760 real images and 770 fake images.


In [ ]:
#Preprocess Dataset Function
def preprocess_images(image_list, size=(128, 128)):
    processed_images = []
    for img in image_list:
        img = img.resize(size)
        img_array = np.array(img) / 255.0
        processed_images.append(img_array)
    return np.array(processed_images)


# Preprocess
real_images_array = preprocess_images(real_images)
fake_images_array = preprocess_images(fake_images)

# Create labels
real_labels = np.ones(len(real_images_array))
fake_labels = np.zeros(len(fake_images_array))

# Combine images and labels
all_images = np.concatenate((real_images_array, fake_images_array), axis=0)
all_labels = np.concatenate((real_labels, fake_labels), axis=0)

# K-Fold Cross Validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)



##Build Models

In [ ]:
# Step 2: AlexNet Implementation (PyTorch)
def build_alexnet_pytorch():
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(4096, 1)  # Binary classification
    return model

def train_alexnet(model, train_loader, val_loader, learning_rate, epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

    return model

# Step 3: VGG16 Implementation (TensorFlow/Keras)
def build_vgg16(learning_rate):
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
    x = Flatten()(base_model.output)
    x = Dense(256, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)  # Binary classification
    model = Model(inputs=base_model.input, outputs=x)

    optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model


##Train and test models

In [ ]:

# Step 4: Training and Testing
fold = 1
for train_val_index, test_index in kf.split(all_images):
    # Split into training/validation and test
    train_val_images = all_images[train_val_index]
    train_val_labels = all_labels[train_val_index]
    test_images = all_images[test_index]
    test_labels = all_labels[test_index]

    # Further split train_val into training and validation (80-20 split)
    val_split = int(len(train_val_images) * 0.2)
    train_images = train_val_images[val_split:]
    train_labels = train_val_labels[val_split:]
    val_images = train_val_images[:val_split]
    val_labels = train_val_labels[:val_split]

    # PyTorch Data Preparation
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    train_dataset = TensorDataset(
        torch.stack([transform(img) for img in train_images]),
        torch.tensor(train_labels, dtype=torch.float32)
    )
    val_dataset = TensorDataset(
        torch.stack([transform(img) for img in val_images]),
        torch.tensor(val_labels, dtype=torch.float32)
    )
    test_dataset = TensorDataset(
        torch.stack([transform(img) for img in test_images]),
        torch.tensor(test_labels, dtype=torch.float32)
    )

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Train AlexNet
    print(f"\nTraining AlexNet for Fold {fold}")
    alexnet_model = build_alexnet_pytorch()
    alexnet_model = train_alexnet(alexnet_model, train_loader, val_loader, learning_rate=0.001)

    # Train VGG16
    print(f"\nTraining VGG16 for Fold {fold}")
    vgg16_model = build_vgg16(learning_rate=0.001)
    vgg16_model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=5, batch_size=32)

    # Testing
    print(f"\nTesting Results for Fold {fold}")
    # AlexNet
    alexnet_model.eval()
    alex_preds, alex_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = torch.sigmoid(alexnet_model(inputs).squeeze())
            preds = (outputs > 0.5).float()
            alex_preds.extend(preds.cpu().numpy())
            alex_labels.extend(labels.cpu().numpy())
    print("AlexNet:")
    print(classification_report(alex_labels, alex_preds))
    print(confusion_matrix(alex_labels, alex_preds))

    # VGG16
    vgg_preds = (vgg16_model.predict(test_images) > 0.5).astype("int32")
    print("VGG16:")
    print(classification_report(test_labels, vgg_preds))
    print(confusion_matrix(test_labels, vgg_preds))

    fold += 1



Training AlexNet for Fold 1
Epoch 1/5, Loss: 0.6512
Epoch 2/5, Loss: 0.4829
Epoch 3/5, Loss: 0.3956
Epoch 4/5, Loss: 0.3286
Epoch 5/5, Loss: 0.2348

Training VGG16 for Fold 1
Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 673s 26s/step - accuracy: 0.6071 - loss: 0.6880 - val_accuracy: 0.5245 - val_loss: 0.6821
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 638s 24s/step - accuracy: 0.7326 - loss: 0.5653 - val_accuracy: 0.5245 - val_loss: 0.9038
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 677s 24s/step - accuracy: 0.8231 - loss: 0.4431 - val_accuracy: 0.6078 - val_loss: 0.7407
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 623s 24s/step - accuracy: 0.8775 - loss: 0.3113 - val_accuracy: 0.4853 - val_loss: 1.2131
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 680s 24s/step - accuracy: 0.8538 - loss: 0.3353 - val_accuracy: 0.7696 - val_loss: 0.4255

Testing Results for Fold 1
AlexNet:
              precision    recall  f1-score   support

         0.0       0.82      0.87      0.84       260
         1.0       0.85      0.80      0.83

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5, Loss: 0.6253
Epoch 2/5, Loss: 0.4916
Epoch 3/5, Loss: 0.4283
Epoch 4/5, Loss: 0.3332
Epoch 5/5, Loss: 0.3104

Training VGG16 for Fold 2
Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 631s 24s/step - accuracy: 0.6368 - loss: 0.6749 - val_accuracy: 0.5980 - val_loss: 0.7006
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 678s 24s/step - accuracy: 0.7597 - loss: 0.5177 - val_accuracy: 0.5245 - val_loss: 0.9887
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 622s 24s/step - accuracy: 0.8536 - loss: 0.3766 - val_accuracy: 0.7745 - val_loss: 0.4662
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 682s 24s/step - accuracy: 0.8852 - loss: 0.2980 - val_accuracy: 0.8039 - val_loss: 0.5022
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 688s 24s/step - accuracy: 0.8811 - loss: 0.2990 - val_accuracy: 0.8382 - val_loss: 0.3311

Testing Results for Fold 2
AlexNet:
              precision    recall  f1-score   support

         0.0       0.62      0.98      0.76       238
         1.0       0.97      0.48      0.64       272

    accuracy     

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5, Loss: 0.6988
Epoch 2/5, Loss: 0.5222
Epoch 3/5, Loss: 0.4072
Epoch 4/5, Loss: 0.3299
Epoch 5/5, Loss: 0.2618

Training VGG16 for Fold 3
Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 625s 24s/step - accuracy: 0.6191 - loss: 0.6694 - val_accuracy: 0.6716 - val_loss: 0.6310
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 685s 24s/step - accuracy: 0.7820 - loss: 0.4990 - val_accuracy: 0.2941 - val_loss: 1.2895
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 626s 24s/step - accuracy: 0.7782 - loss: 0.4468 - val_accuracy: 0.8088 - val_loss: 0.3954
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 625s 24s/step - accuracy: 0.8694 - loss: 0.3008 - val_accuracy: 0.7794 - val_loss: 0.4305
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 720s 26s/step - accuracy: 0.9499 - loss: 0.1722 - val_accuracy: 0.9559 - val_loss: 0.1046

Testing Results for Fold 3
AlexNet:
              precision    recall  f1-score   support

         0.0       0.76      0.97      0.85       272
         1.0       0.94      0.64      0.77       238

    accuracy     